In [0]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Capstone_ML/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Interventions + Property

In [6]:
interv_property_merge1 = pd.read_csv(root_path+'data/0_Cleaned_Datasets/UnitEval_with_Interventions/donneesouvertes-interventions-sim_merged.csv')
interv_property_merge2 = pd.read_csv(root_path+'data/0_Cleaned_Datasets/UnitEval_with_Interventions/donneesouvertes-interventions-sim-2005-2014_merged.csv')
print(interv_property_merge1.shape)
print(interv_property_merge2.shape)
#Concatenate both csvs
interv_property_merge = interv_property_merge1.append(interv_property_merge2)
print(interv_property_merge.shape)

(540121, 21)
(908486, 18)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(1448607, 21)


In [7]:
interv_property_merge.rename(columns={"Creation Date Time": "CREATION_DATE_TIME", 
                                      "Description Groupe": "DESCRIPTION_GROUPE",
                                      "Incident Type Desc": "INCIDENT_TYPE_DESC",
                                     "Incident Nbr": "INCIDENT_NBR",
                                     "Latitude": "LATITUDE",
                                     "Longitude": "LONGITUDE",
                                     "Mtm8 X": "MTM8_X",
                                     "Mtm8 Y": "MTM8_Y"},
                            inplace=True)
list(interv_property_merge)
interv_property_merge.head()

,Avg. Annee Construction,Avg. Etage Hors Sol,Avg. Nombre Logement,Avg. Superficie Batiment,Avg. Superficie Terrain,CIV,Caserne,CREATION_DATE_TIME,DESCRIPTION_GROUPE,Division,INCIDENT_NBR,INCIDENT_TYPE_DESC,LATITUDE,LONGITUDE,MTM8_X,MTM8_Y,Municipality Final,Municipality-ID-Final,Nom Arrond,Nom Ville,Nombre Unites
0,1937.77,1.894802,2.906561,1353.240000,213.490579,1.0,26,1/1/2015 0:03,SANS FEU,11,1,Inondation,45.535695,-73.580573,298507.2,5043978.6,Plateau Mont-Royal,54.0,Plateau Mont-Royal,Montréal,1.0
1,1973.08,1.589981,2.448150,1698.620000,332.610361,NaN,25,1/1/2015 0:05,1-REPOND,10,2,Ac.vÃ©h./1R/s.v./ext/29B/D,45.494526,-73.574328,298990.7,5039403.1,Ville-Marie,52.0,Ville-Marie,Montréal,2.0
2,1966.61,1.765270,2.128760,491.779678,1172.480000,1.0,64,1/1/2015 0:08,Alarmes-incendies,7,3,Appel de Cie de dÃ©tection,45.442000,-73.665779,291831.1,5033576.5,Lachine,88.0,Lachine,Montréal,3.0
3,1983.18,1.543088,1.521679,397.219559,1007.330000,1.0,38,1/1/2015 0:11,1-REPOND,6,4,Premier rÃ©pondant,45.641295,-73.489985,305580.7,5055711.1,Rivière-des-Prairies / Pointe-aux-Trembles,51.0,Rivière-des-Prairies / Pointe-aux-Trembles,Montréal,1.0
4,1958.44,1.820865,3.438996,989.185854,585.483598,1.0,34,1/1/2015 0:14,SANS FEU,9,5,Odeur suspecte - gaz,45.483481,-73.616008,295731.3,5038179.4,Côte-des-Neiges / Notre-Dame-de-Grâce,59.0,Côte-des-Neiges / Notre-Dame-de-Grâce,Montréal,2.0


### Initial Cleanup

In [8]:
print(interv_property_merge.isnull().sum())

#Keep track of nans in CIV, LAT, LONG, MTM8_X, MTM8_Y, DESCRIPTION_GROUPE, NOMBRE_UNITES and CIV

Avg. Annee Construction      29884
Avg. Etage Hors Sol          29884
Avg. Nombre Logement         29884
Avg. Superficie Batiment     29884
Avg. Superficie Terrain      29884
CIV                         967529
Caserne                          0
CREATION_DATE_TIME               6
DESCRIPTION_GROUPE               6
Division                         0
INCIDENT_NBR                     0
INCIDENT_TYPE_DESC               0
LATITUDE                         1
LONGITUDE                        1
MTM8_X                      908486
MTM8_Y                      908486
Municipality Final           29884
Municipality-ID-Final        29884
Nom Arrond                       0
Nom Ville                        0
Nombre Unites                  573
dtype: int64


### Incident Type
INCIDENT_TYPE_DESC: type d'incident (voir fichier joint Incendie Type-description)

In [9]:
incident_type = interv_property_merge['INCIDENT_TYPE_DESC'].unique().tolist()
#print(incident_type)
print('Incident Type:', len(incident_type))

Incident Type: 160


In [10]:
int_type_db = pd.read_csv(root_path+'data/1_Interventions/type-interventions-descriptions20161122.csv')
int_type_db.columns=['INCIDENT_TYPE_DESC', 'Description']   #renaming columns to have a match name between both files to join
int_type_db.head()
print('Incident Type database:', int_type_db.shape)

Incident Type database: (170, 2)


In [0]:
#Must do an inner join in order to remove columns whose incident_type_desc 
#do not match the listed descriptions in the secondary csv file
#Natural/Inner join: To keep only rows that match from the data frames

#interv_property_merge = pd.merge(interv_property_merge, int_type_db, on='INCIDENT_TYPE_DESC', how='inner')
#print(interv_property_merge.shape)

In [0]:
#int_datadate.groupby('INCIDENT_TYPE_DESC').count()[['LATITUDE']].plot.barh()

In [0]:
#print('Largest:\n',int_datajoin['INCIDENT_TYPE_DESC'].value_counts().nlargest(10))
#print('\n Smallest:\n',int_datajoin['INCIDENT_TYPE_DESC'].value_counts().nsmallest(10))

#print(int_datadate['INCIDENT_TYPE_DESC'].count().nlargest(10))

In [14]:
#Factorize this column
interv_property_merge['INCIDENT_TYPE_DESC'], incident_Labels = interv_property_merge.INCIDENT_TYPE_DESC.factorize()
incident_Labels

Index(['Inondation', 'Ac.vÃ©h./1R/s.v./ext/29B/D',
       'Appel de Cie de dÃ©tection', 'Premier rÃ©pondant',
       'Odeur suspecte - gaz', 'Alarme vÃ©rification', '10-22 sans feu',
       'DÃ©chets en feu', 'Aide Ã  un citoyen', 'Ac.vÃ©h./1R/s.v./V.R./29B/D',
       ...
       'Sauv.effon.struc. 10-09 F7/GAZ', 'Acc. sans vict. av. feu-tunnel',
       'AÃ©roport 10-09', 'Sauv.effon.tranchÃ©e 10-09 +CDI', 'AÃ©roport 10-07',
       'Acc. victime feu - pont', 'Acc. victime feu - bÃ¢timent',
       'Feu de cuisson', 'VÃ©rification', 'Sauv.effon.tranchÃ©e'],
      dtype='object', length=160)

### Description Group
DESCRIPTION_GROUPE: regroupement des types d' interventions en catégories : Incendies de bâtiments, Autres incendies, Sans incendie, Alarmes-incendie, Premiers répondants, Fausses alertes/annulations.

In [15]:
descr_grp = interv_property_merge['DESCRIPTION_GROUPE'].unique().tolist()
print('Descritpion Group:', len(descr_grp))
print(descr_grp)

Descritpion Group: 13
['SANS FEU', '1-REPOND', 'Alarmes-incendies', 'AUTREFEU', 'INCENDIE', 'FAU-ALER', 'NOUVEAU', nan, 'Sans incendie', 'Autres incendies', 'Fausses alertes/annulations', 'Premier rÃ©pondant', 'Incendies de bÃ¢timents']


In [16]:
#Remove rows with NOUVEAU
interv_property_merge = interv_property_merge[interv_property_merge['DESCRIPTION_GROUPE'] != 'NOUVEAU']
interv_property_merge = interv_property_merge[interv_property_merge['DESCRIPTION_GROUPE'] != 'nan']
print(interv_property_merge.shape)

(1448606, 21)


In [17]:
#Checking the balance of data for DESCRIPTION_GROUPE
interv_property_merge['DESCRIPTION_GROUPE'].value_counts()

Premier rÃ©pondant             488429
1-REPOND                       367885
Sans incendie                  216607
Alarmes-incendies              205648
SANS FEU                        93149
Autres incendies                38389
Incendies de bÃ¢timents         14593
AUTREFEU                        13407
INCENDIE                         5315
Fausses alertes/annulations      4680
FAU-ALER                          498
Name: DESCRIPTION_GROUPE, dtype: int64

In [0]:
#int_datadrop.head()

#Factorize this column
#int_datadrop['DESCRIPTION_GROUPE'] = int_datadrop.DESCRIPTION_GROUPE.factorize()[0]

In [19]:
#data binning

#Fire: 'Alarmes-incendies', 'INCENDIE', 'Incendies de bâtiments'
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'Alarmes-incendies'] = 1
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'INCENDIE'] = 1
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'Incendies de bÃ¢timents'] = 1
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'Autres incendies'] = 1
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'AUTREFEU'] = 1

#NoFire:'Sans incendie', 'SANS FEU', 'Autres incendies', 'AUTREFEU', 'Fausses alertes/annulations', 'FAU-ALER', 'Premier répondant', '1-REPOND'
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'Sans incendie'] = 0
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'SANS FEU'] = 0
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'Fausses alertes/annulations'] = 0
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'FAU-ALER'] = 0
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == 'Premier rÃ©pondant'] = 0
interv_property_merge.DESCRIPTION_GROUPE[interv_property_merge.DESCRIPTION_GROUPE == '1-REPOND'] = 0


interv_property_merge['DESCRIPTION_GROUPE'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-p

0    1171248
1     277352
Name: DESCRIPTION_GROUPE, dtype: int64

In [20]:
#Checking the balance of data for DESCRIPTION_GROUPE
interv_property_merge['DESCRIPTION_GROUPE'].value_counts()

0    1171248
1     277352
Name: DESCRIPTION_GROUPE, dtype: int64

### Fire Stations
CASERNE: numéro de la caserne responsable du territoire où est survenu l'événement.

In [21]:
stations = interv_property_merge['Caserne'].unique().tolist()
#print(stations)
print('Fire Station', len(stations))

Fire Station 67


In [0]:
#Checking the balance of data for INCIDENT_TYPE_DESC
#int_datadate.groupby('CASERNE').count()[['LATITUDE']].plot.barh()

In [0]:
#print('Largest:\n',int_datadrop['CASERNE'].value_counts().nlargest(10))
#print('\n Smallest:\n',int_datadrop['CASERNE'].value_counts().nsmallest(10))

#Could redo based on fire alarms only

### Date Time
CREATION_DATE_TIME: date et heure de l'événement.

In [24]:
interv_property_merge['CREATION_DATE_TIME'] = pd.to_datetime(interv_property_merge['CREATION_DATE_TIME'])
interv_property_merge['Year'] = pd.DatetimeIndex(interv_property_merge['CREATION_DATE_TIME']).year
interv_property_merge['Month'] = pd.DatetimeIndex(interv_property_merge['CREATION_DATE_TIME']).month
interv_property_merge['Day'] = pd.DatetimeIndex(interv_property_merge['CREATION_DATE_TIME']).day
interv_property_merge['Hour'] = pd.DatetimeIndex(interv_property_merge['CREATION_DATE_TIME']).hour
interv_property_merge['Day_of_week'] = interv_property_merge['CREATION_DATE_TIME'].dt.dayofweek

#Drop 2019 data since incomplete
interv_property_merge = interv_property_merge[interv_property_merge.Year != 2019]

interv_property_merge.head()

,Avg. Annee Construction,Avg. Etage Hors Sol,Avg. Nombre Logement,Avg. Superficie Batiment,Avg. Superficie Terrain,CIV,Caserne,CREATION_DATE_TIME,DESCRIPTION_GROUPE,Division,INCIDENT_NBR,INCIDENT_TYPE_DESC,LATITUDE,LONGITUDE,MTM8_X,MTM8_Y,Municipality Final,Municipality-ID-Final,Nom Arrond,Nom Ville,Nombre Unites,Year,Month,Day,Hour,Day_of_week
0,1937.77,1.894802,2.906561,1353.240000,213.490579,1.0,26,2015-01-01 00:03:00,0,11,1,0,45.535695,-73.580573,298507.2,5043978.6,Plateau Mont-Royal,54.0,Plateau Mont-Royal,Montréal,1.0,2015.0,1.0,1.0,0.0,3.0
1,1973.08,1.589981,2.448150,1698.620000,332.610361,NaN,25,2015-01-01 00:05:00,0,10,2,1,45.494526,-73.574328,298990.7,5039403.1,Ville-Marie,52.0,Ville-Marie,Montréal,2.0,2015.0,1.0,1.0,0.0,3.0
2,1966.61,1.765270,2.128760,491.779678,1172.480000,1.0,64,2015-01-01 00:08:00,1,7,3,2,45.442000,-73.665779,291831.1,5033576.5,Lachine,88.0,Lachine,Montréal,3.0,2015.0,1.0,1.0,0.0,3.0
3,1983.18,1.543088,1.521679,397.219559,1007.330000,1.0,38,2015-01-01 00:11:00,0,6,4,3,45.641295,-73.489985,305580.7,5055711.1,Rivière-des-Prairies / Pointe-aux-Trembles,51.0,Rivière-des-Prairies / Pointe-aux-Trembles,Montréal,1.0,2015.0,1.0,1.0,0.0,3.0
4,1958.44,1.820865,3.438996,989.185854,585.483598,1.0,34,2015-01-01 00:14:00,0,9,5,4,45.483481,-73.616008,295731.3,5038179.4,Côte-des-Neiges / Notre-Dame-de-Grâce,59.0,Côte-des-Neiges / Notre-Dame-de-Grâce,Montréal,2.0,2015.0,1.0,1.0,0.0,3.0


In [25]:
#data binning QUART for merge with crime
#Hour breakdown taken from crime's QUART definition
#jour : Entre 8h01 et 16h
#soir : Entre 16h01 et minuit
#nuit : Entre 00h01 et 8h

conditions = [
    (interv_property_merge['Hour'] >=0 ) & (interv_property_merge['Hour'] < 8 ),
    (interv_property_merge['Hour'] >= 8 ) & (interv_property_merge['Hour'] < 16 ),
    (interv_property_merge['Hour'] >= 16 )]
choices = ['nuit', 'jour', 'soir']
interv_property_merge['QUART'] = np.select(conditions, choices)

interv_property_merge.QUART.value_counts()

jour    576893
soir    541847
nuit    271711
0            6
Name: QUART, dtype: int64

In [26]:
#data binning
#1-Spring - April4, May5, June6
#2-Summer - July7, August8, September9
#3-Fall - October10, November11, December12
#4-Winter - January1, February2, March 3

bins = [-1, 3, 6, 9, 12]
labels = [1,2,3,4]
interv_property_merge['season'] = pd.cut(interv_property_merge['Month'], bins=bins, labels=labels)

interv_property_merge.season.value_counts()

3    361586
4    353950
2    346178
1    328737
Name: season, dtype: int64

In [27]:
#data binning
#Monday=0, Sunday=6.

conditions = [
    (interv_property_merge['Day_of_week'] >=0 ) & (interv_property_merge['Day_of_week'] < 5 ),
    (interv_property_merge['Day_of_week'] >= 5 )]
choices = ['weekDay', 'WeekEnd']
interv_property_merge['weekDayEnd'] = np.select(conditions, choices)

interv_property_merge.weekDayEnd.value_counts()

weekDay    1003497
WeekEnd     386954
0                6
Name: weekDayEnd, dtype: int64

In [28]:
interv_property_merge.head()

,Avg. Annee Construction,Avg. Etage Hors Sol,Avg. Nombre Logement,Avg. Superficie Batiment,Avg. Superficie Terrain,CIV,Caserne,CREATION_DATE_TIME,DESCRIPTION_GROUPE,Division,INCIDENT_NBR,INCIDENT_TYPE_DESC,LATITUDE,LONGITUDE,MTM8_X,MTM8_Y,Municipality Final,Municipality-ID-Final,Nom Arrond,Nom Ville,Nombre Unites,Year,Month,Day,Hour,Day_of_week,QUART,season,weekDayEnd
0,1937.77,1.894802,2.906561,1353.240000,213.490579,1.0,26,2015-01-01 00:03:00,0,11,1,0,45.535695,-73.580573,298507.2,5043978.6,Plateau Mont-Royal,54.0,Plateau Mont-Royal,Montréal,1.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay
1,1973.08,1.589981,2.448150,1698.620000,332.610361,NaN,25,2015-01-01 00:05:00,0,10,2,1,45.494526,-73.574328,298990.7,5039403.1,Ville-Marie,52.0,Ville-Marie,Montréal,2.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay
2,1966.61,1.765270,2.128760,491.779678,1172.480000,1.0,64,2015-01-01 00:08:00,1,7,3,2,45.442000,-73.665779,291831.1,5033576.5,Lachine,88.0,Lachine,Montréal,3.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay
3,1983.18,1.543088,1.521679,397.219559,1007.330000,1.0,38,2015-01-01 00:11:00,0,6,4,3,45.641295,-73.489985,305580.7,5055711.1,Rivière-des-Prairies / Pointe-aux-Trembles,51.0,Rivière-des-Prairies / Pointe-aux-Trembles,Montréal,1.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay
4,1958.44,1.820865,3.438996,989.185854,585.483598,1.0,34,2015-01-01 00:14:00,0,9,5,4,45.483481,-73.616008,295731.3,5038179.4,Côte-des-Neiges / Notre-Dame-de-Grâce,59.0,Côte-des-Neiges / Notre-Dame-de-Grâce,Montréal,2.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay


In [0]:
# Merging with Crime

In [30]:
interv_property_merge['LAT_3sigFig'] = interv_property_merge.LATITUDE.round(3)
interv_property_merge['LONG_3sigFig'] = interv_property_merge.LONGITUDE.round(3)
print(interv_property_merge.shape)
interv_property_merge.head()

(1390457, 31)


,Avg. Annee Construction,Avg. Etage Hors Sol,Avg. Nombre Logement,Avg. Superficie Batiment,Avg. Superficie Terrain,CIV,Caserne,CREATION_DATE_TIME,DESCRIPTION_GROUPE,Division,INCIDENT_NBR,INCIDENT_TYPE_DESC,LATITUDE,LONGITUDE,MTM8_X,MTM8_Y,Municipality Final,Municipality-ID-Final,Nom Arrond,Nom Ville,Nombre Unites,Year,Month,Day,Hour,Day_of_week,QUART,season,weekDayEnd,LAT_3sigFig,LONG_3sigFig
0,1937.77,1.894802,2.906561,1353.240000,213.490579,1.0,26,2015-01-01 00:03:00,0,11,1,0,45.535695,-73.580573,298507.2,5043978.6,Plateau Mont-Royal,54.0,Plateau Mont-Royal,Montréal,1.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay,45.536,-73.581
1,1973.08,1.589981,2.448150,1698.620000,332.610361,NaN,25,2015-01-01 00:05:00,0,10,2,1,45.494526,-73.574328,298990.7,5039403.1,Ville-Marie,52.0,Ville-Marie,Montréal,2.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay,45.495,-73.574
2,1966.61,1.765270,2.128760,491.779678,1172.480000,1.0,64,2015-01-01 00:08:00,1,7,3,2,45.442000,-73.665779,291831.1,5033576.5,Lachine,88.0,Lachine,Montréal,3.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay,45.442,-73.666
3,1983.18,1.543088,1.521679,397.219559,1007.330000,1.0,38,2015-01-01 00:11:00,0,6,4,3,45.641295,-73.489985,305580.7,5055711.1,Rivière-des-Prairies / Pointe-aux-Trembles,51.0,Rivière-des-Prairies / Pointe-aux-Trembles,Montréal,1.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay,45.641,-73.490
4,1958.44,1.820865,3.438996,989.185854,585.483598,1.0,34,2015-01-01 00:14:00,0,9,5,4,45.483481,-73.616008,295731.3,5038179.4,Côte-des-Neiges / Notre-Dame-de-Grâce,59.0,Côte-des-Neiges / Notre-Dame-de-Grâce,Montréal,2.0,2015.0,1.0,1.0,0.0,3.0,nuit,1,weekDay,45.483,-73.616


In [32]:
crime_data = pd.read_csv(root_path+'data/0_Cleaned_Datasets/Cleaned_crime_data.csv')
crime_data.shape

(92962, 10)

In [33]:
interv_crime_property_merge = pd.merge(crime_data, interv_property_merge, 
                              on=['LAT_3sigFig','LONG_3sigFig','Month','Year','QUART'], 
                              how='right')
print(interv_crime_property_merge.shape)

(1390457, 36)


In [34]:
interv_crime_property_merge.rename(columns={"CATEGORIE_fact": "CATEGORIE_TotalCrime"},
                            inplace=True)
interv_crime_property_merge.head()

,Unnamed: 0,LAT_3sigFig,LONG_3sigFig,Month,Year,QUART,CATEGORIE_TotalCrime,CATG_Vol,CATG_Mort,CATG_Autre,Avg. Annee Construction,Avg. Etage Hors Sol,Avg. Nombre Logement,Avg. Superficie Batiment,Avg. Superficie Terrain,CIV,Caserne,CREATION_DATE_TIME,DESCRIPTION_GROUPE,Division,INCIDENT_NBR,INCIDENT_TYPE_DESC,LATITUDE,LONGITUDE,MTM8_X,MTM8_Y,Municipality Final,Municipality-ID-Final,Nom Arrond,Nom Ville,Nombre Unites,Day,Hour,Day_of_week,season,weekDayEnd
0,0.0,45.455,-73.548,2.0,2015.0,soir,25.0,0.0,0.0,25.0,1974.91,1.637366,2.203775,1263.790000,348.532711,1.0,67,2015-02-27 23:45:00,0,5,21538,6,45.454947,-73.547853,301057.3,5035003.0,Verdun,83.0,Verdun,Montréal,5.0,27.0,23.0,4.0,1,weekDay
1,1.0,45.583,-73.572,3.0,2015.0,jour,18.0,0.0,0.0,18.0,1979.04,1.755194,2.727049,465.824564,729.701940,1.0,22,2015-03-07 11:12:00,0,8,24143,3,45.583442,-73.572020,299180.0,5049284.2,Saint-Léonard,85.0,Saint-Léonard,Montréal,1.0,7.0,11.0,5.0,1,WeekEnd
2,5.0,45.506,-73.556,12.0,2018.0,jour,11.0,10.0,0.0,1.0,1973.08,1.589981,2.448150,1698.620000,332.610361,NaN,20,2018-12-05 13:15:00,0,6,119159,3,45.506489,-73.556366,300395.5,5040731.3,Ville-Marie,52.0,Ville-Marie,Montréal,1.0,5.0,13.0,2.0,4,weekDay
3,22.0,45.515,-73.559,7.0,2017.0,nuit,8.0,0.0,0.0,8.0,1973.08,1.589981,2.448150,1698.620000,332.610361,1.0,20,2017-07-03 03:12:00,0,6,65724,3,45.515457,-73.558931,300195.8,5041728.1,Ville-Marie,52.0,Ville-Marie,Montréal,1.0,3.0,3.0,0.0,3,weekDay
4,22.0,45.515,-73.559,7.0,2017.0,nuit,8.0,0.0,0.0,8.0,1973.08,1.589981,2.448150,1698.620000,332.610361,1.0,20,2017-07-15 00:59:00,0,6,70188,3,45.515457,-73.558931,300195.8,5041728.1,Ville-Marie,52.0,Ville-Marie,Montréal,1.0,15.0,0.0,5.0,3,WeekEnd


In [0]:
interv_crime_property_merge.to_csv(root_path+'data/Interventions_Crime_Property_Merge.csv')